### Importing the necessary libraries

In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels
from statsmodels.multivariate.manova import MANOVA

### Loading the dataset

In [13]:
data = sns.load_dataset('iris')
data.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [14]:
data.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')

#### Using the library

In [17]:
dependent_vars = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
independent_vars = 'species'

In [18]:
manova_model = MANOVA.from_formula(f'{"+".join(dependent_vars)} ~ {independent_vars}',data=data)
manova_result = manova_model.mv_test()
print(manova_result)

                   Multivariate linear model
                                                                
----------------------------------------------------------------
       Intercept         Value  Num DF  Den DF   F Value  Pr > F
----------------------------------------------------------------
          Wilks' lambda  0.0170 4.0000 144.0000 2086.7720 0.0000
         Pillai's trace  0.9830 4.0000 144.0000 2086.7720 0.0000
 Hotelling-Lawley trace 57.9659 4.0000 144.0000 2086.7720 0.0000
    Roy's greatest root 57.9659 4.0000 144.0000 2086.7720 0.0000
----------------------------------------------------------------
                                                                
----------------------------------------------------------------
        species          Value  Num DF  Den DF   F Value  Pr > F
----------------------------------------------------------------
          Wilks' lambda  0.0234 8.0000 288.0000  199.1453 0.0000
         Pillai's trace  1.1919 8.0000 290.00

#### Manually

In [31]:
X = data[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].values
y = data['species'].values

# Unique groups
groups = np.unique(y)
groups

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [34]:
# Calculate the mean vectors for each group
mean_vectors = []
for group in groups:
    mean_vectors.append(np.mean(X[y == group], axis=0))
print("Mean vectors:", mean_vectors)
# Calculate the overall mean vector
overall_mean = np.mean(X, axis=0)
print("Overall mean:",overall_mean)

Mean vectors: [array([5.006, 3.428, 1.462, 0.246]), array([5.936, 2.77 , 4.26 , 1.326]), array([6.588, 2.974, 5.552, 2.026])]
Overall mean: [5.84333333 3.05733333 3.758      1.19933333]


In [39]:
# Calculate the within-group scatter matrix
S_W = np.zeros((4, 4))
for group, mean_vec in zip(groups, mean_vectors):
    group_scatter = np.cov(X[y == group].T) * (X[y == group].shape[0] - 1)
    S_W += group_scatter

# Calculate the between-group scatter matrix
S_B = np.zeros((4, 4))
for mean_vec in mean_vectors:
    n = X[y == group].shape[0]
    mean_diff = (mean_vec - overall_mean).reshape(4, 1)
    S_B += n * (mean_diff).dot(mean_diff.T)

In [40]:
# Calculate Wilks' lambda
lambda_val = np.linalg.det(S_W) / np.linalg.det(S_W + S_B)
print(f"Wilks' lambda: {lambda_val}")

Wilks' lambda: 0.02343863065087817


In [35]:
# g1 = data.loc[data['species']=='virginica']
# g2 = data.loc[data['species']=='setosa']
# g3 = data.loc[data['species']=='versicolor']

In [42]:
# print(help(np.linalg))